In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!unzip -qq /kaggle/input/cassava-disease/train.zip

In [3]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt

def preprocessing(image):
    re = cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)
#     hsv = cv2.cvtColor(re, cv2.COLOR_BGR2HSV)
#     green = cv2.inRange(hsv, (42, 16, 16), (232, 256, 256))
#     mask = cv2.bitwise_not(green)
#     masked_image = cv2.bitwise_and(re, re, mask=mask)

    return re

In [4]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
 
category = ['cbb', 'cdsd', 'cgm', 'cmd', 'healty']
data = []
label = []

for dirname, _, filenames in os.walk('train/cbb'):
    for filename in filenames:
        data.append(preprocessing(cv2.imread(os.path.join(dirname, filename))))
        label.append(0)
        
for dirname, _, filenames in os.walk('train/cdsd'):
    for filename in filenames:
        data.append(preprocessing(cv2.imread(os.path.join(dirname, filename))))
        label.append(1)
        
for dirname, _, filenames in os.walk('train/cgm'):
    for filename in filenames:
        data.append(preprocessing(cv2.imread(os.path.join(dirname, filename))))
        label.append(2)
        
for dirname, _, filenames in os.walk('train/cmd'):
    for filename in filenames:
        data.append(preprocessing(cv2.imread(os.path.join(dirname, filename))))
        label.append(3)

for dirname, _, filenames in os.walk('train/healty'):
    for filename in filenames:
        data.append(preprocessing(cv2.imread(os.path.join(dirname, filename))))
        label.append(4)

print('done..')

In [5]:
data = np.array( [img for img in data], dtype='float32')
data = data/255.0

label = np.array(label, dtype='uint8')

print(data.shape, label.shape)

In [6]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.25, random_state=42)
print(train_x.shape, test_x.shape)

In [7]:
# data = []
# label = []

In [8]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D 
from tensorflow.keras.optimizers import Adam

model = Sequential() 
model.add(Input((256, 256, 3)))

model.add(Conv2D(32, (5, 5)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((3,3)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((3,3)))

model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((3,3)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))


model.summary()

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [10]:
model.fit(train_x, train_y, epochs=144, batch_size=32)

In [11]:
test_loss, test_acc = model.evaluate(test_x, test_y, verbose=2)